## Web scraping(웹 스크래핑, 문서 수집하기)
[Open In Colab](https://colab.research.google.com/github/langchain-ai/langchain/blob/v0.1/docs/docs/use_cases/web_scraping.ipynb)

### 4. Research automation
* 구글 검색을 활용한 리서치 자동화
  * [Programmable Search Engine](https://cse.google.com/cse?cx=a61c425e10bee4b8d)
* APIFY를 활용한 리서치 자동화
  * [Integration Tutorial: How to use LangChain with Apify scrapers](https://www.youtube.com/watch?v=zcfeiVdiGJg)

#### 구성도
<img src="./images/web_research.png" width="800">

- [langchain-ai/web-explorer](https://github.com/langchain-ai/web-explorer) 예시 화면

<img src="./images/research_automation01.png" width="600"> <img src="./images/research_automation02.png" width="522">


#### Resources
 - [Custom Search JSON API: 소개](https://developers.google.com/custom-search/v1/introduction?hl=k")



In [8]:
from python_environment_check import check_packages

lib = {
    'streamlit': '1.29.0', 
    'langchain': '0.0.354',
    'chromadb': '0.4.3', 
    'openap': '1.30.1',
    'langchain-openai': '0.1.7',
    'langchain-core': '0.2.0',
    'langchain-chroma': '0.1.1',
    'openai': '1.30.1',
    'google-api-core': '2.11.1',
    'google-api-python-client': '2.95.0',
    'google-auth': '2.22.0',
    'google-auth-httplib2': '0.1.0',
    'googleapis-common-protos': '1.59.1',
    'tiktoken==0.7.0',
    'faiss-cpu==1.8.0',
    'apify-client': '1.7.0', 
#    'beautifulsoup4': '4.12.2',
    'html2text': '2024.2.26',
    'dotenv': '0.0.5',
    'nest_asyncio': '1.6.0',
}

#check_packages(lib)

[OK] Your Python version is 3.8.19 (default, Mar 20 2024, 19:55:45) [MSC v.1916 64 bit (AMD64)]


In [1]:
import dotenv
dotenv.load_dotenv('envls')

# 미리 실행해야 함
import nest_asyncio
nest_asyncio.apply()

# 미리 설치해야 함
#!playwright install


In [26]:
import os
#print(os.environ['GOOGLE_CSE_ID'])
#print(os.environ['GOOGLE_API_KEY'])

### 구글 검색을 활용한 리서치 자동화
#### 구글 Custom Search Engine 생성 및 키 필요
<img src="./images/googlecse01.png" width="800">
<img src="./images/googlecse02.png" width="800">
<img src="./images/googlecse03.png" width="800">
<img src="./images/googlecse04.png" width="800">
<img src="./images/googlecse05.png" width="800">
<img src="./images/googlecse06.png" width="800">
<img src="./images/googlecse07.png" width="800"> 

In [27]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain_chroma import Chroma
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [28]:
# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(), persist_directory="./db/chroma_db_oai"
)

# LLM
llm = ChatOpenAI(temperature=0)

# Search
search = GoogleSearchAPIWrapper()

In [29]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

In [31]:
# Run
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
from langchain.chains import RetrievalQAWithSourcesChain

#user_input = "How do LLM Powered Autonomous Agents work?"
#user_input = "고래대학교의 학생수는?"
user_input = "고려대학교의 학생수는?"

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)
result = qa_chain({"question": user_input})
result


INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': '고래대학교의 학생수는?', 'text': LineList(lines=['1. 고래대학교의 총 학생수는?\n', '2. 고래대학교에는 몇 명의 학생이 재학 중인가요?\n', '3. 고래대학교의 학생 인원은 얼마나 되나요?'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. 고래대학교의 총 학생수는?\n', '2. 고래대학교에는 몇 명의 학생이 재학 중인가요?\n', '3. 고래대학교의 학생 인원은 얼마나 되나요?']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Search results: [{'title': '아이비리그 대학들 중 규모가 가장 작은 대학, 다트머스대학교 ...', 'link': 'https://blog.naver.com/princetonreview_?Redirect=Log&logNo=221587859092', 'snippet': 'Jul 17, 2019 ... Dartmouth College의 시작은 미국 인디언 교육이었으나 현재 다트머스대학교 본 캠퍼스의 총 인디언 학생 수는 전체 학생 수의 1% 정도이며, 부유층\xa0...'}]
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retri

{'question': '고래대학교의 학생수는?',
 'answer': '고려대학교의 서울 캠퍼스에는 2023년 기준으로 학부생이 20,373명, 대학원생이 9,947명, 교원이 4,408명, 직원이 1,152명이 있습니다. 세종 캠퍼스에는 2023년 기준으로 학부생이 6,574명, 대학원생이 927명, 교원이 765명, 직원이 223명이 있습니다.\n',
 'sources': 'https://namu.wiki/w/%EA%B3%A0%EB%A0%A4%EB%8C%80%ED%95%99%EA%B5%90'}

#### * APIFY를 활용한 리서치 자동화
#### [APIFY](https://apify.com/)
* 2015년에 설립된 프라하 기반 스타트업
* 대량의 웹 데이터를 수집 및 분석하고 웹 프로세스를 자동화하는 오픈 소스 도구 제공
* Apify Store를 운영하며, 필요한 스크래퍼를 등록하여 사용할 수 있음

<img src="./images/apify00.png" width="800">
<img src="./images/apify01.png" width="800">
<img src="./images/apify05.png" width="800">
<img src="./images/apify02.png" width="800">
<img src="./images/apify03.png" width="800">
<img src="./images/apify04.png" width="800">

In [16]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.docstore.document import Document
from langchain_community.utilities import ApifyWrapper
from langchain_community.document_loaders import ApifyDatasetLoader

apify = ApifyWrapper()
# Call the Actor to obtain text from the crawled webpages
"""
1. 수집기를 활용한 방법
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    #actor_id="astronomical_lizard/naver-blog-scraper", 
    #run_input={"startUrls": [{"url": "/docs/integrations/chat/"}]},
    run_input={"startUrls": [{"url": "https://docs.apify.com/academy/web-scraping-for-beginners"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["full_text"] or "", metadata={"source": item["url"]}
    ),
)
"""
# 2. 수집한 데이터 베이스를 활용한 방법
loader = ApifyDatasetLoader(
    #dataset_id="mCkg6zDcTEO5JYX2F", 
    dataset_id="rneQUybW8o0bUwZpK", 
    dataset_mapping_function=lambda item: Document(
        page_content=item["full_text"] or "", metadata={"source": item["url"]}
    ),
)

# Create a vector store based on the crawled data
index = VectorstoreIndexCreator().from_loaders([loader])

# Query the vector store
#query = "Are any OpenAI chat models integrated in LangChain?"
query = "Inspection에 대해서 알려주세요"
result = index.query(query)
print(result)


 Inspection은 제품, 시설, 문서 등을 세밀하게 조사하거나 검토하는 과정을 의미합니다. 이는 제품의 품질이나 안전성을 확인하기 위해 수행되며, 제품의 결함이나 문제점을 발견하고 수정하는데 사용될 수 있습니다. Inspection은 제품 생산 과정에서 중요한 역할을 하며, 제품의 품질을 보장하기 위해 필수적인 과정입니다.


## 결론
- 서비스하기에는 속도가 느리다. (개선할 부분을 찾는 것이 숙제)
- 제공되는 Transformer의 세부적인 옵션이 필요하다.
  - BeautifulSoup 라이브러리를 사용하는 것이 더 효과적일 수 있다.
- 상용 검색엔진과 연동하면 더 효과적으로 서비스할 수 있을 것 같다.
- 스크래퍼의 변환 결과와 LLM의 스키마 및 프롬프트 결과를 위한 충분한 테스트 필요

